In [23]:
using PyPlot

function S_k_omega(lambda_range, lambda_in, theta, A, T_e,T_i,n_e,Z, v_fi=0, v_fe=0)
    #    Returns a normalised spectral density function.
    #    Implements the model of Sheffield (2nd Ed.)
    #    One ion, one electron species with independent temeperatures
    #    Electron velocity is with respect to ion velocity
    #    Returns S(k,w) for each wavelength in lambda_range assuming
    #    input wavelength lambda_in. Both in metres
    #    Theta is angle between k_in and k_s in degrees
    #    A i atomic mass, Z is ion charge
    #    T_e, T_i in eV, n_e in cm^-3
    #    V_fi and V_fe in m/s

    m_p=1.66e-27
    m_e=9.11e-31
    c=3e8
    e_charge=1.6e-19
    RW_and_Iw(z)=1-z.*(2*dawson(z)-im*sqrt(pi)*exp(-z.^2))
    #physical parameters
    m_i=m_p*A
    om_pe=5.64e4*n_e^0.5#electron plasma frequency
    #define omega and k as in Sheffield 113
    ki=2*pi/lambda_in
    omega_i=((c*ki)^2+om_pe^2)^0.5

    ks=2*pi./lambda_range
    omega_s=((c*ks).^2+om_pe^2).^0.5
    
    th=theta/180.0*pi#convert to radians for cosine function
    k=(ks.^2+ki^2-2*ks*ki*cos(th)).^0.5
    omega=omega_s-omega_i
    
    #define dimensionless parameters ala Sheffield
    a=sqrt(2*e_charge*T_e/m_e)
    b=sqrt(2*e_charge*T_i/m_i)
    x_e=(omega./k+v_fe+v_fi)/a
    x_i=(omega./k+v_fi)/b
    lambda_De=7.43*(T_e/n_e)^0.5 #Debeye length in m
    #the all important alpha parameter
    alpha2=(1./(k*lambda_De)).^2
    #set up the Fadeeva function
    chi_i=alpha2.*(Z*T_e/T_i).*(RW_and_Iw(x_i)) #ion susceptibility
    chi_e=alpha2.*(RW_and_Iw(x_e))#electron susceptibility
    epsilon=1+chi_e+chi_i#dielectric function
    fe0=1/(sqrt(pi)*a)*exp(-x_e.^2)#electron Maxwellian function
    fi0=1/(sqrt(pi)*b)*exp(-x_i.^2)#ion Maxwellian
    skw=2*pi./k.*(abs(1-chi_e./epsilon).^2).*fe0+Z*(abs(chi_e./epsilon).^2).*fi0
    skw./maximum(skw)
end

S_k_omega (generic function with 3 methods)

In [2]:
l=linspace(530e-9, 534e-9, 1000)
l0=532e-9
theta=45
A=12
n_e=5e17
T_e=100
T_i=100
Z=6

6

In [25]:
@timeit S_k_omega(l, l0, theta, A, T_e,T_i,n_e,Z)

1000 loops, best of 3: 805.61 µs per loop


0.00080561037

In [10]:
using TimeIt

In [82]:
x=1
chi_w(x), chi_erfi(x), chi_daw(x)

(0.604103223382251 + 0.0im,0.3479506678267078 - 1.0761590138255368im,-0.0761590138255368 + 0.6520493321732922im)